# GenreMatch
Data cleaning and model generation  
*Jeremy Freedman, Reza Madhavan, Kunal Sheth*

In [1]:
!unzip Archive.zip

unzip:  cannot find or open Archive.zip, Archive.zip.zip or Archive.zip.ZIP.


In [2]:
!ls utils

azlyrics.py  grab_genius.ipynb	__pycache__
genius.py    grab_lyrics.py	songlyrics.py


In [3]:
import pandas as pd
import numpy as np
import utils.songlyrics as sl
from collections import defaultdict
from nltk.corpus import stopwords as sw
from nltk import download as nltk_download
import sklearn as sk
from sklearn import tree
import collections
import gc
import math

In [4]:
a=pd.read_csv('data/song_lyrics.csv').dropna()
new_df = a.copy()
head = new_df.head(5)

In [5]:
# download the NLTK stopwords list, if necessary
nltk_download('stopwords')

[nltk_data] Downloading package stopwords to /home/jerem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# we need to clean and update the NLTK stopwords for our data
# we're stripping out punctuation entirely, which the stopwords are not equipped to handle
# could optionally add music-centric stopwords ('oh', 'yeah', 'like', etc)

temp_words = sw.words('english')
stopwords = []
additions = ['im', 'ill', 'id', 'oh', 'cant', 'ive']
for w in temp_words:
    stopwords.append(sl._clean(w))
stopwords += additions
print(stopwords)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', '

In [7]:
for index,row in new_df.iterrows():
  if index % 100 == 0:
    print(index)

  song = row['Lyrics']
  x=song.find('\n')
  song2 = sl._clean(song[x+1:])
  new_song = ''
  for i in range(len(song2)):
    char = song2[i]

    if char == '\n':
      new_song += ' '
    elif char == '(' or char == ')':
      new_song += ''
    else:
      new_song += char


    new_song = new_song.encode('ascii','ignore').decode()

  song2 = new_song.split(' ')
  song2 = list(filter(lambda x : x!='' and x not in stopwords, song2))
  song2 = song2[:len(song2)-1]

  new_df.at[index, 'Lyrics'] = song2


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000


In [8]:
wordfreq = []
word_appearance_cnt = defaultdict(int) # how many songs contain word W? used for IDF
for index,row in new_df.iterrows():
  lyrics = row['Lyrics']
  x = dict(collections.Counter(lyrics))
  for w in x:
    word_appearance_cnt[w] += 1
  # mapped = []
  # for k in x.keys():
  #   mapped.append((k,x[k]))
  wordfreq.append(x)

new_df['WordFreq'] = wordfreq

In [9]:
new_df['UniqueWords']= new_df.apply(lambda row : len(row['WordFreq']), axis = 1)
new_df

,Unnamed: 0,Artist,Song,Lyrics,Genre,WordFreq,UniqueWords
0,0,One Direction,Perfect,"[might, never, knight, shinin, armor, might, n...",pop,"{'might': 4, 'never': 4, 'knight': 1, 'shinin'...",71
1,1,One Direction,Story of My Life,"[written, walls, stories, explain, leave, hear...",pop,"{'written': 3, 'walls': 2, 'stories': 1, 'expl...",68
2,2,One Direction,Drag Me Down,"[got, fire, heart, scared, dark, never, seen, ...",pop,"{'got': 4, 'fire': 2, 'heart': 2, 'scared': 2,...",37
3,3,One Direction,What Makes You Beautiful,"[insecure, know, turning, heads, walk, door, n...",pop,"{'insecure': 1, 'know': 18, 'turning': 1, 'hea...",54
4,4,One Direction,Night Changes,"[going, tonight, changes, something, red, moth...",pop,"{'going': 2, 'tonight': 3, 'changes': 8, 'some...",65
...,...,...,...,...,...,...,...
4074,4074,Def Leppard,Action! Not Words,"[sick, tired, damn, tv, gonna, make, movie, wa...",metal,"{'sick': 1, 'tired': 1, 'damn': 1, 'tv': 1, 'g...",61
4075,4075,Def Leppard,Rock Rock (Till You Drop),"[ready, get, set, tear, place, apart, need, ti...",metal,"{'ready': 1, 'get': 2, 'set': 1, 'tear': 1, 'p...",74
4076,4076,Def Leppard,Slang,"[sitting, dark, getting, taken, said, somethin...",metal,"{'sitting': 1, 'dark': 1, 'getting': 1, 'taken...",41
4077,4077,Def Leppard,Die Hard the Hunter,"[lets, welcome, home, soldier, boy, far, away,...",metal,"{'lets': 2, 'welcome': 2, 'home': 2, 'soldier'...",65


In [10]:
uniq_words = set()
for x in new_df['WordFreq']:
  uniq_words = uniq_words.union(x.keys())
uniq_wordict = {k : 0 for k in uniq_words}

In [11]:
word_df = new_df[['Genre', 'WordFreq', 'Artist', 'UniqueWords']]

for index,row in word_df.iterrows():
  if index % 100 == 0:
    print(f'\r{100 * index // len(new_df)}%', end='')
  for k in row['WordFreq']:
    # uniq_wordict[k] = row['WordFreq'][k] # original (raw count)
    uniq_wordict[k] = row['WordFreq'][k] * math.log(len(new_df) / word_appearance_cnt[k]) # tf-idf

  word_df.at[index, 'WordFreq'] = uniq_wordict
  word_df.at[index, 'Artist'] = list(uniq_wordict.values()) + [row['UniqueWords']]
  uniq_wordict = {k : 0 for k in uniq_words}

word_df = word_df.replace({'Artist':'WordFreqList'})


98%

In [12]:
# free up memory
gc.collect()

0

In [13]:
word_df = word_df.rename(columns = {'Artist':'WordFreqList'})
print(word_df)

      Genre                                           WordFreq  \
0       pop  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
1       pop  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
2       pop  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
3       pop  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
4       pop  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
...     ...                                                ...   
4074  metal  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
4075  metal  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
4076  metal  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
4077  metal  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   
4078  metal  {'pittsfield': 0, 'imitating': 0, 'babes': 0, ...   

                                           WordFreqList  UniqueWords  
0     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...           71  
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...          

In [14]:
# make a 80/20 train/test split of lyric words tagged by origin genre
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(list(word_df['WordFreqList']), list(word_df['Genre']), test_size=0.2)

print(f'Split ratio: {len(X_test) / (len(X_test) + len(X_train))}\nTest set: {len(X_test)}\nTrain set: {len(X_train)}')

Split ratio: 0.20009808729769496
Test set: 816
Train set: 3262


In [15]:
X_train[1][-1]

74

In [16]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)

DecisionTreeClassifier()

In [17]:
preds = clf.predict(X_test)

In [18]:
np.sum(preds == Y_test)/len(preds)

0.3737745098039216




---
# STOP